# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('output_data_file')
weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed
0,0,Cocorit,MX,27.58,-109.97,52.57,67,0,5.26
1,1,Laguna,US,38.42,-121.42,59.00,47,1,3.36
2,2,Rocha,UY,-34.48,-54.33,58.50,95,0,3.04
3,3,Ushuaia,AR,-54.80,-68.30,53.60,71,75,11.41
4,4,Busselton,AU,-33.65,115.33,91.99,41,100,5.01
...,...,...,...,...,...,...,...,...,...
553,553,Poum,NC,-20.23,164.02,82.36,86,99,18.39
554,554,Winnemucca,US,40.97,-117.74,35.01,58,1,4.70
555,555,Bagdarin,RU,54.43,113.60,3.58,60,0,7.87
556,556,Kiama,AU,-34.68,150.87,80.01,100,90,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

lat_long = weather_df[['Latitude', 'Longitude']]
humidity = weather_df['Humidity']

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=300, point_radius=5 )
fig.add_layer(heat_layer)

In [4]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed
0,0,Cocorit,MX,27.58,-109.97,52.57,67,0,5.26
1,1,Laguna,US,38.42,-121.42,59.00,47,1,3.36
2,2,Rocha,UY,-34.48,-54.33,58.50,95,0,3.04
3,3,Ushuaia,AR,-54.80,-68.30,53.60,71,75,11.41
4,4,Busselton,AU,-33.65,115.33,91.99,41,100,5.01


In [6]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


vacay_df = weather_df.loc[(weather_df['High Temp'] < 80 ) & (weather_df['High Temp'] > 70 ) 
                          & (weather_df['Wind Speed'] < 10 ) & (weather_df['Cloudiness'] == 0 )].dropna()



vacay_df

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed
141,141,Springbok,ZA,-29.66,17.89,73.90,27,0,5.93
152,152,Saint-Louis,SN,16.33,-15.00,71.60,28,0,2.24
160,160,Karwar,IN,14.80,74.13,78.17,64,0,7.09
226,226,Tura,IN,25.52,90.22,77.76,31,0,6.11
287,287,Bubaque,GW,11.28,-15.83,78.12,64,0,7.74
349,349,Porto-Novo,BJ,6.50,2.60,78.80,78,0,2.24
350,350,Barinas,VE,8.62,-70.21,77.04,49,0,5.44
381,381,Orocue,CO,4.79,-71.34,78.21,51,0,5.32
411,411,Kruisfontein,ZA,-34.00,24.73,70.29,69,0,7.87
441,441,Bahia Honda,CU,22.90,-83.16,72.48,89,0,4.54


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacay_df.copy()

hotel_df['Hotel Name'] = ""

hotel_df

,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
141,141,Springbok,ZA,-29.66,17.89,73.90,27,0,5.93,
152,152,Saint-Louis,SN,16.33,-15.00,71.60,28,0,2.24,
160,160,Karwar,IN,14.80,74.13,78.17,64,0,7.09,
226,226,Tura,IN,25.52,90.22,77.76,31,0,6.11,
287,287,Bubaque,GW,11.28,-15.83,78.12,64,0,7.74,
349,349,Porto-Novo,BJ,6.50,2.60,78.80,78,0,2.24,
350,350,Barinas,VE,8.62,-70.21,77.04,49,0,5.44,
381,381,Orocue,CO,4.79,-71.34,78.21,51,0,5.32,
411,411,Kruisfontein,ZA,-34.00,24.73,70.29,69,0,7.87,
441,441,Bahia Honda,CU,22.90,-83.16,72.48,89,0,4.54,


In [9]:
# geocoordinates

params = {
   
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    params["location"] = f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    response_json = response.json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = response_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("None")
    

# set up a parameters dictionary


# base url

hotel_df
    

# run a request using our params dictionary


None


,Unnamed: 0,City,Country,Latitude,Longitude,High Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
141,141,Springbok,ZA,-29.66,17.89,73.90,27,0,5.93,Faithful Guest Lodge
152,152,Saint-Louis,SN,16.33,-15.00,71.60,28,0,2.24,
160,160,Karwar,IN,14.80,74.13,78.17,64,0,7.09,Netalco Connects
226,226,Tura,IN,25.52,90.22,77.76,31,0,6.11,Hotel RIKMAN Continental
287,287,Bubaque,GW,11.28,-15.83,78.12,64,0,7.74,Casa Dora
349,349,Porto-Novo,BJ,6.50,2.60,78.80,78,0,2.24,Tour Eiffel Hotel Benin
350,350,Barinas,VE,8.62,-70.21,77.04,49,0,5.44,Hotel Suites Mastranto
381,381,Orocue,CO,4.79,-71.34,78.21,51,0,5.32,Hotel Brisas de San Miguel
411,411,Kruisfontein,ZA,-34.00,24.73,70.29,69,0,7.87,Oyster Bay House Rental
441,441,Bahia Honda,CU,22.90,-83.16,72.48,89,0,4.54,Casa Gualberto - Deysi


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map